In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
CREATE OR REPLACE PROCEDURE run_workflow()
RETURNS STRING
LANGUAGE JAVASCRIPT
EXECUTE AS CALLER
AS $$
try {
    // Step 1: Run Data Ingestion
    var result1 = snowflake.execute("RUN NOTEBOOK 'AIRLINEDELAY.PUBLIC.DATA_INGESTION/versions/live/notebook_app.ipynb'");
    
    // Step 2: Run Data Transformation
    var result2 = snowflake.execute("RUN NOTEBOOK 'AIRLINEDELAY.PUBLIC.DATA_TRANSFORMATION/versions/live/notebook_app.ipynb'");
    
    // Step 3: Run Airline Delay EDA
    var result3 = snowflake.execute("RUN NOTEBOOK 'AIRLINEDELAY.PUBLIC.AIRLINEDELAY_EDA/versions/live/notebook_app.ipynb'");
    
    // Step 4: Run Flight Delay Classification Model
    var result4 = snowflake.execute("RUN NOTEBOOK 'AIRLINEDELAY.PUBLIC.FLIGHTDELAY_CLASSIFICATION_MODEL/versions/live/notebook_app.ipynb'");
    
    // Step 5: Run Flight Delay Prediction Model
    var result5 = snowflake.execute("RUN NOTEBOOK 'AIRLINEDELAY.PUBLIC.FLIGHTDELAY_PREDICTION_MODEL/versions/live/notebook_app.ipynb'");
    
    return 'Workflow completed successfully!';
} catch (err) {
    return 'Error during workflow: ' + err.message;
}
$$;


In [ ]:
CREATE OR REPLACE TASK run_workflow_task
WAREHOUSE = COMPUTE_WH  -- Specify the warehouse to use
SCHEDULE = 'USING CRON 0 0 * * * UTC'  -- Runs daily at 12 AM (midnight) UTC
COMMENT = 'Task to run the full workflow procedure.'
AS
 CALL run_workflow();


In [ ]:
ALTER TASK run_workflow_task RESUME;


In [ ]:
SHOW TASKS LIKE 'run_workflow_task';
